In [3]:
import json
import winsound as sd
import threading
from modules.detect import Detector
from modules.crop import crop_image

import torch
#ImageFromVideo.get_image('data/video/2023032319.mov')
import cv2

import time
import requests
# Initialize a class instance

In [4]:
human_detector = Detector()
#dumping_classifier = Classifier()


YOLOR  31daac8 torch 1.12.1 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 8192.0MB)



Fusing layers... 


Model Summary: 306 layers, 36905341 parameters, 36905341 gradients


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


c:\Users\user\anaconda3\lib\site-packages\torch\_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:482.)
  return self._grad
c:\Users\user\anaconda3\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [14]:
def Capture():
    
    cap = cv2.VideoCapture(1)
    print("width:", cap.get(3), "height : ", cap.get(4))
    
    if not cap.isOpened():
        print("Camera open failed!")
        exit()
    
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    delay = round(1000/fps)
    print("fps =" , fps, "delay = ", delay)
    out = cv2.VideoWriter('../action/output1.mp4', fourcc, fps, (w,h))

    if not out.isOpened():
        print('File open failed!')
        cap.release()
        exit()
    
    start_time = time.time()
    
    detected = False
    filenum = 'output1.mp4'
    
    while(True):
        ret, frame = cap.read()
        
        if ret:
            image = cv2.flip(frame, 1)
            
            if not detected:
                with torch.no_grad():
                    det = human_detector.detect(image)
            
                for i, (*xyxy, conf, cls) in enumerate(reversed(det)):
                    if cls == 0:
                        print("human detected")
                        detected = True
                        start_time = time.time()
                        print(start_time)
            
            if detected == True:
                out.write(image)
                                    
                current_time = time.time()
                elapsed_time = current_time - start_time
                if elapsed_time >= 2.0:
                    start_time = current_time
                    print("video saved and reset")
                    out.release()
                    
                    request = threading.Thread(target=send_request, args=filenum[6])
                    request.start()
                
                    if filenum == 'output1.mp4':
                        out = cv2.VideoWriter('../action/output2.mp4', fourcc, fps, (w,h))
                        filenum = 'output2.mp4'
                    elif filenum == 'output2.mp4':
                        out = cv2.VideoWriter('../action/output1.mp4', fourcc, fps, (w,h))
                        filenum = 'output1.mp4'
                        
                    detected = False
                    
                    
                    
            cv2.imshow("VIDEO", image)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    
def send_request(filenum):
    request = {'filenum' : f'output{filenum}.mp4'}
    
    headers = {
    "Content-Type": "application/json"
    }
    url = "http://localhost:8080/tensor" 
                    
    data_json = json.dumps(request)
                    
    r = requests.post(url, data=data_json, headers=headers)
                    
    if r.status_code == 200:
        print("post succeed", r.content.decode('utf-8'))
    else:
        print("post failed", r.status_code)
    return
    



In [17]:
Capture()

width: 640.0 height :  480.0
fps = 30.0 delay =  33
human detected
1693409693.1683679
video saved and reset
post succeed {"dumping": 0.6503480076789856, "walking": 0.3496519923210144}
human detected
1693409703.5077045
video saved and reset
human detected
1693409705.5671353
video saved and reset
post succeed {"dumping": 0.5783114433288574, "walking": 0.42168858647346497}
human detected
1693409708.431386
post succeed {"dumping": 0.811511218547821, "walking": 0.18848875164985657}
video saved and reset
human detected
1693409710.4997225
video saved and reset
post succeed {"dumping": 0.704838752746582, "walking": 0.29516127705574036}
human detected
1693409713.8941429
post succeed {"dumping": 0.3099752962589264, "walking": 0.6900246739387512}
video saved and reset
human detected
1693409715.9659142
video saved and reset
human detected
1693409718.0377336
post succeed {"dumping": 0.23064720630645752, "walking": 0.7693528532981873}
video saved and reset
post succeed {"dumping": 0.0894991159439086